# Artificial Neural Network (ANN) in Raw Numpy

This demo provides very basic Numpy implementations of the following:

1. A feed forward NN represented as a doubly linked list
2. A text classifier that uses a CBOW feature representation
3. Autograd

*Note: This demo conveys some of the underlying constructs used in deep learning packages, not the low level implementation details of those constructs.*

### GPU acceleration

The [CuPy](https://cupy.dev/) Python package allows you to execute Numpy and Scipy methods on a GPU using the Numpy API. If running on a CUDA enabled GPU you can set `device="cuda"`, else you'll need to set `device="cpu"`.

In [1]:
import numpy as np

device = 'cpu' # <---- Set to 'cuda' if running on GPU

if device == 'cuda':
    import cupy as cp
    M = 100000
else:
    cp = np
    M = 10000

### Load DBPedia14 dataset

In [ ]:
import datasets
import pandas as pd

df = datasets.load_dataset(
    'dbpedia_14', 
    split=['train[:100%]', 'test']
)[0].to_pandas().sample(frac=1).reset_index(drop=True)[:M]

K = len(set(df.label))
one_hot_labels = np.eye(K)

### Preprocessing

In [3]:
# NLTK downloads
try:
    from nltk.corpus import stopwords
    sw_nltk = stopwords.words('english')
except Exception:
    import nltk
    import ssl
    try:
        _create_unverified_https_context = ssl._create_unverified_context
    except AttributeError:
        pass
    else:
        ssl._create_default_https_context = _create_unverified_https_context
    nltk.download()

In [4]:
import re
from tqdm import tqdm
from typing import List

email_re = r"""(?:[a-z0-9!#$%&'*+/=?^_`{|}~-]+(?:\.[a-z0-9!#$%&'*+/=?^_`{|}~-]+)
    *|"(?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21\x23-\x5b\x5d-\x7f]
    |\\[\x01-\x09\x0b\x0c\x0e-\x7f])*")@(?:(?:[a-z0-9](?:[a-z0-9-]*[a-z0-9])?\.)+[a-z0-9]
    (?:[a-z0-9-]*[a-z0-9])?|\[(?:(?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.){3}
    (?:25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?|[a-z0-9-]*[a-z0-9]:
    (?:[\x01-\x08\x0b\x0c\x0e-\x1f\x21-\x5a\x53-\x7f]|\\[\x01-\x09\x0b\x0c\x0e-\x7f])+)\])"""

replace = [
    ("<[^>]*>", " "),
    (email_re, " "),                           # Matches emails
    (r"(?<=\d),(?=\d)", ""),                   # Remove commas in numbers
    (r"[*\^\.$&@<>,\-/+{|}=?#:;'\"\[\]]", ""), # Punctuation and other junk
    (r"[\n\t\r]", " "),                        # Removes newlines, tabs, creturn
    (r"[^\x00-\x7F]+", ""),                    # Removes non-ascii chars
    (r"\\+", " "),                             # Removes double-backslashs
    (r"\s+n\s+", " "),                         # 'n' leftover from \\n
    (r"\s+", " "),                             # Strips extra whitespace
        (r"\d+", "DIGIT")                      # Map digits to special token <numbr>
]


def transform(doc: str):
    for repl in replace:
        doc = re.sub(repl[0], repl[1], doc.lower())
    doc = " ".join([w for w in doc.split(" ") if not w in sw_nltk])
    return doc

In [ ]:
from tqdm import tqdm

print(df.content[300])

with tqdm(total=M) as bar:
    for i, content in enumerate(df.content.tolist()):
        df.loc[i, 'content'] = transform(content)
        bar.update(1)

print(df.content[300])

### Featurizer

Below we will build a BOW featurizer, and use it as both a CBOW featurizer and as an embedding lookup

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vocab_size = len(set(" ".join(df.content.tolist()).split(" ")))
print(vocab_size)
featurizer = CountVectorizer(max_features=vocab_size, min_df=4, stop_words=None)
featurizer.fit(df.content.tolist());
featurizer.get_idx = {word: idx for idx, word in enumerate(featurizer.get_feature_names_out())}
featurizer.get_word = {idx: word for idx, word in enumerate(featurizer.get_feature_names_out())}

N = len(featurizer.get_idx)
N

### Features

In [7]:
def get_features(dataframe):
    # Labels
    Y = dataframe.label.to_numpy()
    # One-hot features (just word index pointers)
    X_idxs = [[featurizer.get_idx[word] for word in doc.split(" ") if featurizer.get_idx.get(word) is not None] 
              for doc in dataframe.content]
    # BOW features
    X = np.zeros(shape=[len(dataframe), N], dtype=int)
    for i, seq in enumerate(X_idxs):
        for idx in seq:
            X[i, idx] += 1
    return X, Y

### Data spit

In [8]:
M_tr = int(0.8 * M)
M_cv = int(0.1 * M)
M_te = M - M_tr - M_cv

X_tr, Y_tr = get_features(df[:M_tr])
X_cv, Y_cv = get_features(df[M_tr: M_tr + M_cv])
X_te, Y_te = get_features(df[-M_te:])

del df

In [ ]:
[featurizer.get_word[idx] for idx, c in enumerate(X_tr[0]) if c]

## Neural network as a graph

Deep learning packages implement neural networks as a graph, where each node represents a layer within the network, and each edge represents a transformation that maps one layer onto the next. The two most important components of any deep learning library are (i) efficient implementations of the various transformations used in deep learning (graph edges) that are optimized for various hardware (x86, ARM, NVIDIA GPUS, TPUs etc..), and (ii) automatic differentiation (autograd) which enables users to use the package constructs (linear transformations, activations, loss functions, optimizers etc.) without ever having to implement (or even think about) backpropogation. Below is a (very rudimentary) implementation of the basic computational units of a feed forward network.

*Note: There are two numpy multiplication operations being used here: 1) `@` for matrix multiplication, and 2) `*` for the element-wise (aka Hadamard, aka Schur) product*

### Cross entropy loss

In [10]:
def cross_entropy(P, y):
    ce = cp.sum(-cp.asarray(y) * cp.log(P), axis=1)
    return cp.mean(ce)

### Metrics

In [11]:
def compute_accuracy(Y_hat, Y):
    acc = np.equal(Y_hat, Y).mean()
    return acc

### Numerically stable softmax

In [12]:
def softmax(Z):
    Z = cp.exp(Z - cp.max(Z, axis=1, keepdims=True))
    partition = cp.sum(Z, axis=1, keepdims=True)
    return Z / partition

### Weight initializer

Weight initialization is crucially important in deep learning, if not done properly your network will just diverge after the first gradient update. There are many options here, but it primarily boils down to the distribution you draw values from. A great choice is *Xavier* initialization, named after it's author Xavier Glorot, which draws weights from a normal distribution $W_{ij} \sim N(\mu=0, \sigma^2=\frac{1}{n})$, where $W \in \mathbb{R}^{n \times \cdot}$. For large weight matrices, this is a means to control the variance of the output, $z$, at each layer.

In [13]:
def weight_init(*dims):
    """ Xavier initialization """
    return (1 / cp.sqrt(dims[0])) * cp.random.randn(*dims)

### Dropout module

Dropout is one of the most important regularization techniques in deep learning. In each activation layer during training, a random subset (chosen with probability `prob`) of the activations are masked out (zeroed). On an intuitive and somewhat handwavy level, you can think of this as a way to prevent the network over relying on on a small subset of the pathways through the network, and helps force all of the nodes, and therefore weights, to be utilized (quasi) equally. By doing this, we are scaling the values of the output layer; in order to keep the distribution over each layer consistent during the training and test phases, this means we must apply a scaling factor either during test time ($\frac{1}{prob}$, traditional dropout), or during training ($\frac{1}{1-prob}$, inverted dropout). Inverted dropout is the preferred method as it incurs no runtime cost during inference, while only adding marginal runtime overhead during training (remember, the vast majority of the computation during training is from backpropogation). This is an implementation of inverted dropout.

In [14]:
def apply_dropout(a: cp.ndarray, prob: float):
    mask = cp.random.choice([0.0, 1.0], size=a.shape, p=[prob, 1 - prob])
    a *= mask / (1 - prob)
    return a

### Graph node

The doubly linked list is a natural data structure for implementing networks. Below we make a class called `Node` which will serve as the base class for the various layer types in our network.

Each `Node` contains four things: 
1. Memory to store a layer (`Node.out`)
2. Method to map the parent node's output layer to its output (`Node.out = Node.forward(Node.last.out)`)
3. Method to backpropogate gradients from its child node to its parent node (`Node.error = Node.backward(Node.next.error)`)
4. Methods to perform gradient updates to the parameters that it owns (`Node.update()`, `Node.zero_grad()`)

In [15]:
class Node:
    
    """ 
    Node in computational graph (doubly linked list)
    Note: not to be confused with a node in a network layer
    
    Attributes:
    last: Node        # Parent
    next: Node        # Child
    out: np.ndarray   # Output layer
    error: np.ndarray # dCE/d`out`
    dim: int          # layer dimension
    lr: float         # learning rate
    """
    
    def __init__(self, last, dim: int, lr: float = None):
        self.last = last
        if self.last:
            self.last.next = self
        self.next = None
        self.out = None
        self.error = None
        self.dim = dim
        self.lr = lr
        
    def forward(self, dropout: bool = False):
        pass
    
    def backward(self):
        pass
    
    def zero_grad(self):
        pass
    
    def update(self):
        pass

### Input embedding node

This is an embedding lookup, which will be the first layer in our network. The forward method accepts a batch of BOW features and computes their inner-product with an embedding lookup table. This is an implementation of the continuous *bag-of-features*, in which we we simply sum the embedding for each word in the the BOW input, weighted by their frequency counts.

In [16]:
class InputEmbedding(Node):
    
    def __init__(self, vocab_size, embedding_dim, lr, wt_decay):
        super().__init__(None, embedding_dim, lr)
        self.W = weight_init(embedding_dim, vocab_size)
        self.b = weight_init(1, self.dim)
        self.wt_decay = wt_decay
        self.zero_grad()
    
    def forward(self, X, dropout):
        self.input = cp.asarray(X)
        self.out = self.input @ self.W.T + self.b
        if self.next:
            self.next.forward(dropout)
    
    def zero_grad(self):
        self.W_grad = np.zeros_like(self.W)
        self.b_grad = np.zeros_like(self.b)
    
    def update(self):
        n = self.out.shape[0]
        l1_grad = self.wt_decay * np.sign(self.W)
        l2_grad = self.wt_decay * self.W
        self.W -= self.lr * ((1 / n) * self.W_grad + l1_grad + l2_grad)
        self.b -= self.lr * (1 / n) * self.b_grad
    
    def backward(self, error=None):
        if error is None:
            error = self.next.error
        dW = self.input
        db = 1
        self.W_grad += error.T @ dW
        self.b_grad += cp.sum(error, axis=0) * db

### Leaky relu activation

The leaky relu (rectified linear unit) is a popular activation function (non-linearity).

$LReLU(z) = \max(z, \alpha z) \quad \text{where} \quad \alpha \in (0, 1)$

In [17]:
class LReLU(Node):
    
    def __init__(self, last, alpha=0.001, dropout=0.5):
        super().__init__(last, last.dim)
        self.alpha = alpha
        self.dropout = dropout
    
    def forward(self, dropout):
        self.out = cp.where(self.last.out >= 0, 
                            self.last.out, 
                            self.alpha * self.last.out)
        if dropout:
            self.out = apply_dropout(self.out, prob=self.dropout)
        self.next.forward(dropout)
        
    def update(self):
        self.last.update()
        
    def zero_grad(self):
        self.last.zero_grad()
        
    def backward(self):
        da = cp.where(self.last.out >= 0, 1, self.alpha)
        self.error = self.next.error * da
        self.last.backward()

### Note on the *forward* and *backward* pass

In the above cell you'll note that the `.forward()` method contains a call to `Node.next.forward()`; this is what allows data to flow from input layer to output layer. Also notice that the `.backward()`, `.update()`, and `.zero_grad()` methods all contain calls to either/both its child (`Node.next`) or parent (`Node.last`) along with code that computes gradents and updates parameter weights; this is very basic implementation of *autograd*. These features allow us to simply call `head_node.forward(some_input)` to compute the *forward pass*, and `tail_node.backward()` to compute the *backward pass* in the training loop at the bottom of this notebook.

### Linear node

This is the *linear* unit, sometimes referred to as a `dense` layer. It's just a batched implementation of $XW^T + b$.

In [18]:
class Linear(Node):
    
    def __init__(self, last, dim: int, lr: float, wt_decay: float, has_bias: bool = True):
        super().__init__(last, dim, lr)
        self.has_bias = has_bias
        self.W = weight_init(self.dim, self.last.dim)
        if has_bias:        
            self.b = weight_init(1, self.dim)
        self.wt_decay = wt_decay
        self.zero_grad()
    
    def zero_grad(self):
        self.W_grad = cp.zeros_like(self.W)
        if self.has_bias:
            self.b_grad = cp.zeros_like(self.b)
        self.error = None
        self.last.zero_grad()
    
    def update(self):
        n = self.out.shape[0]
        l2_grad = self.wt_decay * self.W
        self.W -= self.lr * ((1 / n) * self.W_grad + l2_grad)
        if self.has_bias:
            self.b -= self.lr * (1 / n) * self.b_grad
        self.last.update()
    
    def forward(self, dropout):
        self.out = self.last.out @ self.W.T
        if self.has_bias:
            self.out += self.b
        if self.next:
            self.next.forward(dropout=dropout)
        
    def backward(self, error=None):
        if error is None:
            error = self.next.error
        dW = self.last.out
        self.W_grad += error.T @ dW
        if self.has_bias:
            db = 1
            self.b_grad += np.sum(error, axis=0) * db
        self.error = error @ self.W
        self.last.backward()

## Network wrapper

This is a wrapper around our doubly linked list, it allows us to define one object for interacting with the network rather than having to keep track of the *head* and *tail* nodes of our network.

In [19]:
class Network:
    
    input: Node
    last: Node
    probs: np.ndarray
    pred: np.ndarray
    
    def set_lr(self, lr):
        node = self.input
        while node is not None:
            node.lr = lr
            node = node.next
            
    @property
    def lr(self):
        return self.last.lr
        
    @property
    def out(self):
        return self.last.out
    
    def forward(self, x, dropout=False):
        self.input.forward(x, dropout=dropout)
        self.probs = softmax(self.out)
        self.pred = cp.argmax(self.probs, axis=1)
    
    def backward(self, Y):
        loss = self.probs - cp.asarray(Y)
        self.last.backward(loss)
        
    def update(self):
        self.last.update()
        
    def zero_grad(self):
        self.last.zero_grad()

In [20]:
class ANN(Network):
    def __init__(self, vocab_size, layer_size, output_dim, num_layers, lr, wt_decay, alpha, dropout):
        super().__init__()
        self.input = InputEmbedding(vocab_size, layer_size, lr, wt_decay)
        self.last = LReLU(self.input, alpha=alpha, dropout=dropout)
        for i in range(num_layers - 1):
            z = Linear(self.last, layer_size, lr, wt_decay)
            self.last = LReLU(z, dropout=dropout)
        self.last = Linear(self.last, output_dim, lr, wt_decay, has_bias=False)

### Training hyperparameters

In [21]:
alpha = 0.001
lr_init = 0.008
lr_min = 0.0001
lr_decay = 0.985
wt_decay = 5e-5
dropout = 0.1
layer_size = 384
layers = 2
batch_size = 16
batch_size_inference = 1024
epochs = 50

### Network definition

In [22]:
net = ANN(N, layer_size, K, layers, lr=lr_init, wt_decay=wt_decay, alpha=alpha, dropout=dropout)

### Evaluator

In [23]:
def evaluate(net, X, Y, batch_sz):
    m = len(X)
    ce, acc = 0, 0
    for i in range(0, m, batch_sz):
        x = X[i: i + batch_sz]
        y = Y[i: i + batch_sz]
        net.forward(x)
        ce += (x.shape[0] / m) * cross_entropy(net.probs, one_hot_labels[y])
        acc += (x.shape[0] / m) * compute_accuracy(net.pred, y)
    return ce, acc

## Training loop

In [ ]:
shuffle_idx = np.arange(M_tr)

with tqdm(total=epochs * (M_tr // batch_size)) as bar:    
    
    for epoch in range(epochs):
        
        # Randomize data ordering
        np.random.shuffle(shuffle_idx)
        X_shfd = X_tr[shuffle_idx]
        Y_shfd = Y_tr[shuffle_idx]
        
        for i in range(0, M_tr, batch_size):
            
            # Get batch of x,y pairs
            x = X_shfd[i: i + batch_size]
            y = Y_shfd[i: i + batch_size]
            
            # Skip empty slices
            if not x.shape[0]:
                continue
            
            # Forward pass
            net.forward(x, dropout=True)
            
            # Backward pass
            # Note: From lecture 05, remember that the derivative of the
            # cross entropy loss w.r.t. the input (z) to the softmax P(z)
            # is dCE/dz = P(z) - y.
            net.backward(one_hot_labels[y])
            
            # Apply gradient updates
            net.update()
            
            # Zero the stored gradients
            net.zero_grad()
            
            bar.update(1)
        
        # Evaluate performance on training set w/o dropout
        ce_tr, acc_tr = evaluate(net, X_tr, Y_tr, batch_size_inference)
        ce_cv, acc_cv = evaluate(net, X_cv, Y_cv, batch_size_inference)

        bar.set_description("Epoch: %d, lr: %.5f, CE (train/val): %.4f / %.4f, Acc (train/val): %.4f / %.4f" 
                            % (epoch + 1, net.lr, ce_tr, ce_cv, acc_tr, acc_cv))

        # Learning rate annealing
        net.set_lr(max(lr_min, lr_decay * net.lr))

### Compute test performance

Your test accuracy will depend on the sample size $M$.

|Sample size | Test accuracy |
|:--:|:--:|
| $M=10^4$| $\sim 92\%$ |
| $M=10^5$| $\sim 98\%$ |

In [ ]:
ce_te, acc_te = evaluate(net, X_te, Y_te, batch_size_inference)
print(f"Test CE: {ce_te:.4}, Test Acccuracy: {acc_te}")